# 🔧 AI Telco Troubleshooting Challenge - Track 3

**Question**: Can our fine-tuned LLM detect and explain unseen network failures? In other words, for the Track 3: Can we build a specialised edge-cloud LLM to troubleshoot network faults?

**Target**: Enhance the accuracy of Qwen2.5-1.5B-Instruct when answering telco troubleshooting questions in telelogs data.



**Objective**: Train a lightweight LLM to diagnose 5G network throughput issues from drive-test data  

### Installation

In [ ]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    !pip install --upgrade -qqq uv
    !uv pip install vllm==0.11.2 unsloth-zoo unsloth
    !uv pip install transformers==4.56.2
    !uv pip install --no-deps trl==0.22.2

### Unsloth

Load up our `Qwen 2.5 1.5B Instruct` variant(yehoshua00/Qwen2.5-RCA-1.5B), and set parameters

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 8192 # Can increase for longer reasoning traces
lora_rank = 16 # 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "yehoshua00/Qwen2.5-RCA-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 42,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
ERROR 01-23 11:56:26 [fa_utils.py:64] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 01-23 11:56:48 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.56.2. vLLM: 0.11.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Standby mode is enabled. However your setting of `gpu_memory_utilization` will OOM.
Changing `gpu_memory_utilization` to 0.8075.
Unsloth: vLLM loading yeho

/usr/local/lib/python3.12/dist-packages/pydantic/type_adapter.py:605: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `enum` - serialized value may not be as expected [field_name='mode', input_value=3, input_type=int])
  return self.serializer.to_python(


INFO 01-23 11:57:20 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 01-23 11:57:20 [model.py:1745] Using max model len 8192
INFO 01-23 11:57:22 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=4096.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'fp4', 'bnb_4bit_use_double_quant': False, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': [], 'llm_int8_threshold': 6.0}


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2026.1.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage and customize the [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) template for data prep and all reward functions.

In [ ]:
import re
from datasets import load_dataset, Dataset
import os
from google.colab import userdata

# ────────────────────────────────────────────────────────────────
#  System / Format Prompts
# ────────────────────────────────────────────────────────────────

SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
\boxed{...}
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
\boxed{{answer}}
</answer>
"""

# ────────────────────────────────────────────────────────────────
#  Extraction Helpers
# ────────────────────────────────────────────────────────────────

def extract_xml_answer(text: str) -> str:
    if "<answer>" not in text or "</answer>" not in text:
        return ""
    answer_part = text.split("<answer>", 1)[-1].split("</answer>", 1)[0]
    return answer_part.strip()

def extract_boxed_number(text: str) -> str | None:
    if "\\boxed{" not in text:
        return None
    # Take content between last \boxed{ and }
    boxed = text.split("\\boxed{")[-1].split("}")[0]
    cleaned = boxed.strip()
    # Normalize "C2" → "2", "2" → "2"
    if cleaned.startswith("C") and len(cleaned) == 2 and cleaned[1].isdigit():
        return cleaned[1]
    if cleaned.isdigit() and 1 <= int(cleaned) <= 8:
        return cleaned
    return None

# ────────────────────────────────────────────────────────────────
#  Load Dataset from Hugging Face
# ────────────────────────────────────────────────────────────────

def get_telelogs_dataset(split: str = "train", token: str | None = None) -> Dataset:
    """
    Load netop/TeleLogs from Hugging Face.

    Requirements:
      - You must accept the dataset conditions on https://huggingface.co/datasets/netop/TeleLogs
      - Provide a HF read token (Settings → Access Tokens) if gated
    """
    if token is None:
        token = userdata.get("TELCO_HF_TOKEN")  # or hard-code during dev (not recommended)

    ds = load_dataset(
        "netop/TeleLogs",
        split=split,
        token=token,                # pass token if gated
    )

    # Map to the prompt / answer format your trainer expects
    def format_example(example):
        # Assuming columns include at least 'question' and 'answer'
        # If column names differ → adjust here (e.g. 'input', 'label', 'root_cause')
        question = example.get("question", "")
        answer   = example.get("answer",   "")

        # Normalize answer to just the digit "1"–"8"
        clean_answer = answer.upper().replace("C", "") if answer.startswith(("C","c")) else answer
        clean_answer = clean_answer.strip()

        return {
            "prompt": [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user",   "content": question}
            ],
            "answer": clean_answer   # "1", "2", ..., "8"
        }

    ds = ds.map(format_example, remove_columns=ds.column_names)
    return ds


# ────────────────────────────────────────────────────────────────
#  Reward Functions (adapted for C1–C8 → digit 1–8)
# ────────────────────────────────────────────────────────────────

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    questions = [p[-1]['content'][:100] + "..." for p in prompts]  # truncate for logging

    extracted = [extract_boxed_number(extract_xml_answer(r)) for r in responses]

    # Debug print (can be removed in production)
    for q, a, r, ext in zip(questions, answer, responses, extracted):
        print("-"*30)
        print(f"Q: {q}")
        print(f"GT: {a}")
        print(f"Resp: {r[:120]}...")
        print(f"Extracted: {ext}")

    return [2.0 if ext == gt else 0.0 for ext, gt in zip(extracted, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    extracted = [extract_boxed_number(extract_xml_answer(c[0]['content'])) for c in completions]
    return [0.5 if x and x.isdigit() and 1 <= int(x) <= 8 else 0.0 for x in extracted]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n\\boxed\{[1-8]\}\n</answer>\n$"
    responses = [c[0]["content"] for c in completions]
    matches = [bool(re.match(pattern, r, re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>.*<answer>.*\\boxed\{[1-8]\}.*</answer>"
    responses = [c[0]["content"] for c in completions]
    matches = [bool(re.search(pattern, r, re.DOTALL)) for r in responses]
    return [0.5 if m else 0.0 for m in matches]

def count_xml(text: str) -> float:
    score = 0.0
    if text.count("<reasoning>\n") == 1: score += 0.125
    if text.count("\n</reasoning>\n") == 1: score += 0.125
    if text.count("\n<answer>\n") == 1: score += 0.125
    if text.count("\n</answer>") == 1: score += 0.125
    if "\\boxed{" in text and text.count("\\boxed{") == 1: score += 0.125
    # small penalty for trailing junk
    if text.split("\n</answer>")[-1].strip(): score -= 0.02
    return score

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    return [count_xml(c[0]["content"]) for c in completions]


# ────────────────────────────────────────────────────────────────
#  Usage Example
# ────────────────────────────────────────────────────────────────

if __name__ == "__main__":
    # Replace with your real token or set HF_TOKEN env var
    dataset = get_telelogs_dataset(split="train", token=None)

    print(f"Loaded dataset: {len(dataset)} examples")
    print(dataset[0])  # inspect first formatted example

README.md:   0%|          | 0.00/7.33k [00:00<?, ?B/s]

test.json:   0%|          | 0.00/3.93M [00:00<?, ?B/s]

troubleshooting/train.json:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/864 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Loaded dataset: 2400 examples
{'answer': '2', 'prompt': [{'content': '\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n\x08oxed{...}\n</answer>\n', 'role': 'system'}, {'content': "Analyze the 5G wireless network drive-test user plane data and engineering parameters.\nIdentify the reason for the throughput dropping below 600Mbps in certain road sections.\nFrom the following 8 potential root causes, select the most likely one and enclose its number in \\boxed{{}} in the final answer.\n\nC1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.\nC2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.\nC3: A neighboring cell provides higher throughput.\nC4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.\nC5: Frequent handovers degrade performance.\nC6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.\nC7: Test vehicle speed exceeds 40km/h,

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [ ]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4, # Increase to 4 for smoother training (default 1)
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 8196,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` * `gradient_accumulation_steps` * `world_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,400 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)


WARNING 01-23 12:06:01 [abstract.py:312] Executor is not sleeping.
WARNING 01-23 12:06:01 [abstract.py:312] Executor is not sleeping.
WARNING 01-23 12:06:01 [processor.py:246] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.
INFO 01-23 12:06:46 [abstract.py:306] It took 0.136126 seconds to fall asleep.
Unsloth: Will smartly offload gradients to save VRAM!
------------------------------
Q: Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the ...
GT: 2
Resp: \boxed{C3}...
Extracted: None
------------------------------
Q: Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the ...
GT: 2
Resp: \boxed{C8} 

The most likely root cause leading to the throughput dropping below 600Mbps is that the average 

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / xmlcount_reward_func / mean,rewards / xmlcount_reward_func / std,rewards / soft_format_reward_func / mean,rewards / soft_format_reward_func / std,rewards / strict_format_reward_func / mean,rewards / strict_format_reward_func / std,rewards / int_reward_func / mean,rewards / int_reward_func / std,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std
1,-0.000000,0.120625,0.151972,158.531250,7.000000,200.000000,0.750000,34.125000,7.000000,174.000000,0.000000,0.073750,0.070995,0.000000,0.000000,0.000000,0.000000,0.046875,0.148072,0.000000,0.000000
2,-0.000000,0.058125,0.075987,161.812500,7.000000,200.000000,0.750000,47.250000,7.000000,160.000000,0.000000,0.058125,0.076134,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.042500,0.061279,162.812500,7.000000,200.000000,0.750000,51.250000,7.000000,130.000000,0.000024,0.042500,0.063500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.069844,0.047343,167.000000,7.000000,200.000000,0.718750,82.666664,7.000000,198.000000,0.000022,0.069844,0.057100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.038594,0.060099,175.593750,7.000000,200.000000,0.781250,88.428574,7.000000,188.000000,0.000026,0.038594,0.063376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.046406,0.065224,177.531250,7.000000,200.000000,0.843750,56.200001,7.000000,156.000000,0.000023,0.046406,0.063376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.046406,0.072143,164.531250,7.000000,200.000000,0.750000,58.125000,7.000000,174.000000,0.000022,0.046406,0.070884,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.077656,0.049849,165.906250,7.000000,200.000000,0.812500,18.166668,7.000000,74.000000,0.000021,0.077656,0.052502,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.062031,0.079690,159.468750,7.000000,200.000000,0.718750,55.888889,7.000000,169.000000,0.000026,0.062031,0.081625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.038594,0.065224,174.187500,7.000000,200.000000,0.781250,82.000000,7.000000,175.000000,0.000052,0.038594,0.063376,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Streaming output truncated to the last 5000 lines.
Q: Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the ...
GT: 1
Resp: \boxed{C4} 

The most likely root cause for the throughput dropping below 600 Mbps in certain road sections is due to no...
Extracted: None
------------------------------
Q: Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the ...
GT: 1
Resp: \boxed{C8}

The throughput dropping below 600Mbps in certain road sections can primarily be attributed to the vehicle's ...
Extracted: None
------------------------------
Q: Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the ...
GT: 1
Resp: \boxed{C8} 

Explanation:
The throughput dropped below 600Mbps when the average scheduled RBs were below 160. This scena...
Extracted: None
------------------------------
Q: Analyze the 5G wireless network drive-test user plane data and engineering p

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : """
Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the reason for the throughput dropping below 600Mbps in certain road sections.
From the following 8 potential root causes, select the most likely one and enclose its number in \boxed{{}} in the final answer.

C1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.
C2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.
C3: A neighboring cell provides higher throughput.
C4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.
C5: Frequent handovers degrade performance.
C6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.
C7: Test vehicle speed exceeds 40km/h, impacting user throughput.
C8: Average scheduled RBs are below 160, affecting throughput.

Given:
- The default electronic downtilt value is 255, representing a downtilt angle of 6 degrees. Other values represent the actual downtilt angle in degrees.

Beam Scenario and Vertical Beamwidth Relationships:
- When the cell's Beam Scenario is set to Default or SCENARIO_1 to SCENARIO_5, the vertical beamwidth is 6 degrees.
- When the cell's Beam Scenario is set to SCENARIO_6 to SCENARIO_11, the vertical beamwidth is 12 degrees.
- When the cell's Beam Scenario is set to SCENARIO_12 or above, the vertical beamwidth is 25 degrees.

User plane drive test data as follows：

Timestamp|Longitude|Latitude|GPS Speed (km/h)|5G KPI PCell RF Serving PCI|5G KPI PCell RF Serving SS-RSRP [dBm]|5G KPI PCell RF Serving SS-SINR [dB]|5G KPI PCell Layer2 MAC DL Throughput [Mbps]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 Filtered Tx BRSRP [dBm]|5G KPI PCell Layer1 DL RB Num (Including 0)
2025-05-07 15:23:52.000000|128.188169|32.579273|1|712|-77.0|15.93|1351.25|258|71|284|-|-|-90.49|-95.82|-102.2|-|-|210.91
2025-05-07 15:23:53.000000|128.18814|32.579223|2|71|-80.97|6.6|366.57|258|712|129|-|-|-88.72|-93.26|-105.47|-|-|211.23
2025-05-07 15:23:54.000000|128.188117|32.579174|16|71|-85.5|1.81|334.0|258|712|-|-|-|-84.41|-96.46|-|-|-|210.99
2025-05-07 15:23:55.000000|128.188103|32.579113|14|71|-88.21|5.4|431.94|712|258|-|-|-|-88.04|-93.72|-|-|-|195.26
2025-05-07 15:23:56.000000|128.188088|32.579075|19|71|-78.45|13.59|566.34|712|258|284|-|-|-87.97|-98.7|-105.92|-|-|209.0
2025-05-07 15:23:57.000000|128.188073|32.579033|22|258|-77.45|11.45|1308.74|712|71|284|129|-|-89.56|-93.31|-104.18|-110.78|-|204.95
2025-05-07 15:23:58.000000|128.188051|32.579002|16|258|-81.66|10.06|1310.14|712|71|284|129|-|-84.85|-93.3|-102.87|-114.09|-|210.13
2025-05-07 15:23:59.000000|128.188073|32.578995|26|258|-79.13|16.32|1023.36|712|71|284|129|-|-86.75|-93.71|-104.01|-107.26|-|208.06
2025-05-07 15:24:00.000000|128.188036|32.578984|33|258|-74.09|19.8|770.83|712|71|284|129|-|-89.29|-96.4|-100.21|-114.11|-|207.07
2025-05-07 15:24:01.000000|128.188036|32.578984|25|258|-71.09|19.62|1014.75|712|71|-|-|-|-90.14|-92.93|-|-|-|205.57


Engeneering parameters data as follows：

gNodeB ID|Cell ID|Longitude|Latitude|Mechanical Azimuth|Mechanical Downtilt|Digital Tilt|Digital Azimuth|Beam Scenario|Height|PCI|TxRx Mode|Max Transmit Power|Antenna Model
0034038|4|128.188273|32.579129|50|6|4|0|DEFAULT|5.0|712|64T64R|34.9|NR AAU 2
0033164|27|128.217634|32.58072|240|3|10|0|DEFAULT|29.7|71|32T32R|34.9|NR AAU 1
0034038|2|128.188282|32.579099|140|10|4|0|DEFAULT|5.0|258|64T64R|34.9|NR AAU 2
0034038|24|128.18769|32.57791|40|44|9|10|SCENARIO_9|56.0|284|64T64R|34.9|NR AAU 2
0033164|14|128.189481|32.581043|150|20|255|0|DEFAULT|28.7|129|64T64R|34.9|NR AAU 2
    """},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : """
Analyze the 5G wireless network drive-test user plane data and engineering parameters.
Identify the reason for the throughput dropping below 600Mbps in certain road sections.
From the following 8 potential root causes, select the most likely one and enclose its number in \boxed{{}} in the final answer.

C1: The serving cell's downtilt angle is too large, causing weak coverage at the far end.
C2: The serving cell's coverage distance exceeds 1km, resulting in over-shooting.
C3: A neighboring cell provides higher throughput.
C4: Non-colocated co-frequency neighboring cells cause severe overlapping coverage.
C5: Frequent handovers degrade performance.
C6: Neighbor cell and serving cell have the same PCI mod 30, leading to interference.
C7: Test vehicle speed exceeds 40km/h, impacting user throughput.
C8: Average scheduled RBs are below 160, affecting throughput.

Given:
- The default electronic downtilt value is 255, representing a downtilt angle of 6 degrees. Other values represent the actual downtilt angle in degrees.

Beam Scenario and Vertical Beamwidth Relationships:
- When the cell's Beam Scenario is set to Default or SCENARIO_1 to SCENARIO_5, the vertical beamwidth is 6 degrees.
- When the cell's Beam Scenario is set to SCENARIO_6 to SCENARIO_11, the vertical beamwidth is 12 degrees.
- When the cell's Beam Scenario is set to SCENARIO_12 or above, the vertical beamwidth is 25 degrees.

User plane drive test data as follows：

Timestamp|Longitude|Latitude|GPS Speed (km/h)|5G KPI PCell RF Serving PCI|5G KPI PCell RF Serving SS-RSRP [dBm]|5G KPI PCell RF Serving SS-SINR [dB]|5G KPI PCell Layer2 MAC DL Throughput [Mbps]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 PCI|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 1 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 2 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 3 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 4 Filtered Tx BRSRP [dBm]|Measurement PCell Neighbor Cell Top Set(Cell Level) Top 5 Filtered Tx BRSRP [dBm]|5G KPI PCell Layer1 DL RB Num (Including 0)
2025-05-07 15:23:52.000000|128.188169|32.579273|1|712|-77.0|15.93|1351.25|258|71|284|-|-|-90.49|-95.82|-102.2|-|-|210.91
2025-05-07 15:23:53.000000|128.18814|32.579223|2|71|-80.97|6.6|366.57|258|712|129|-|-|-88.72|-93.26|-105.47|-|-|211.23
2025-05-07 15:23:54.000000|128.188117|32.579174|16|71|-85.5|1.81|334.0|258|712|-|-|-|-84.41|-96.46|-|-|-|210.99
2025-05-07 15:23:55.000000|128.188103|32.579113|14|71|-88.21|5.4|431.94|712|258|-|-|-|-88.04|-93.72|-|-|-|195.26
2025-05-07 15:23:56.000000|128.188088|32.579075|19|71|-78.45|13.59|566.34|712|258|284|-|-|-87.97|-98.7|-105.92|-|-|209.0
2025-05-07 15:23:57.000000|128.188073|32.579033|22|258|-77.45|11.45|1308.74|712|71|284|129|-|-89.56|-93.31|-104.18|-110.78|-|204.95
2025-05-07 15:23:58.000000|128.188051|32.579002|16|258|-81.66|10.06|1310.14|712|71|284|129|-|-84.85|-93.3|-102.87|-114.09|-|210.13
2025-05-07 15:23:59.000000|128.188073|32.578995|26|258|-79.13|16.32|1023.36|712|71|284|129|-|-86.75|-93.71|-104.01|-107.26|-|208.06
2025-05-07 15:24:00.000000|128.188036|32.578984|33|258|-74.09|19.8|770.83|712|71|284|129|-|-89.29|-96.4|-100.21|-114.11|-|207.07
2025-05-07 15:24:01.000000|128.188036|32.578984|25|258|-71.09|19.62|1014.75|712|71|-|-|-|-90.14|-92.93|-|-|-|205.57


Engeneering parameters data as follows：

gNodeB ID|Cell ID|Longitude|Latitude|Mechanical Azimuth|Mechanical Downtilt|Digital Tilt|Digital Azimuth|Beam Scenario|Height|PCI|TxRx Mode|Max Transmit Power|Antenna Model
0034038|4|128.188273|32.579129|50|6|4|0|DEFAULT|5.0|712|64T64R|34.9|NR AAU 2
0033164|27|128.217634|32.58072|240|3|10|0|DEFAULT|29.7|71|32T32R|34.9|NR AAU 1
0034038|2|128.188282|32.579099|140|10|4|0|DEFAULT|5.0|258|64T64R|34.9|NR AAU 2
0034038|24|128.18769|32.57791|40|44|9|10|SCENARIO_9|56.0|284|64T64R|34.9|NR AAU 2
0033164|14|128.189481|32.581043|150|20|255|0|DEFAULT|28.7|129|64T64R|34.9|NR AAU 2
    """},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
